# Variational Monte Carlo

## 1D Harmonic Oscillator

To get started with the variational Monte Carlo Method, we start with a 1D example: The Harmonic oscillator. 
We start with the trial wavefunction 
$$\psi(x) = e^{-\alpha x^2}$$

For a fixed $\alpha$, we can calculate $\langle E(\alpha) \rangle = \langle \psi |H|\psi\rangle$ by performing the integral
$$\frac{\int_{-\infty}^{\infty}\psi^*(x)H\psi(x)}{\langle \psi | \psi \rangle} \mathrm{d}x$$

We approximate this integral using the variational Monte Carlo Method. 
$$\int_{-\infty}^{\infty}\frac{\psi^2(x)}{\langle \psi | \psi \rangle}\frac{H\psi(x)}{\psi(x)} \mathrm{d}x $$

This first part of the expression gives a probability density, while the second part is called 'the local energy', which we can calculate analytically in the case of the 1D Harmonic Oscillator. 
This probability density gives immediately a sampling function for performing the integral! For high values of the probability, we want more sampling points, to improve the accuracy of our integral estimate. 
To indeed sample our Monte Carlo points according to this position we use random Walkers. 

Random walkers:
At each step, they can shift to a new position. The probability. distribution gives via a ratio the probability that this step is indeed taken. This leads to a sample of points, distributed with the desired probability distribution!!

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
%matplotlib inline

### Initialize Parameters and random numbers

In [4]:
Nsteps = 30000     #Number of walker steps
N = 400            #Number of random walkers
alpha_1D = [0.40, 0.45, 1/2, 0.55, 0.60]   #Various values of alpha

x_init_HO = np.random.uniform(-1, 1, N) #Unif. Initial position Random Walker
step_equi = 4000   #Number of equilibrium steps

#Matrix with random numbers, to accept (or not) the step
accept_matrix_HO = np.random.uniform(size = (N, Nsteps))  
#Matrix to give random stepsizes
step_matrix_HO = np.random.uniform(-1, 1, size = (N, Nsteps))  

#Maximal stepsize
d_max1D = 0.2

### Calculate the Energy, using Variational Monte Carlo

We calculate the expectation value of the energy and the and also show the variance of the Energy. We compare the values with the book by Jos Thijssen. 

In [7]:
print('alpha      E      Var( E )')

#We change the parameter in the wavefunction 
for alpha in alpha_1D:
    
    #We set the initial position of the walkers to be random
    Pos_walkers = np.zeros((N, Nsteps+1))
    Pos_walkers[:,0] = np.array(x_init_HO)
    
    #For each timestep we do the following:
    for i in range(Nsteps):
       
        Pos_old = Pos_walkers[:,i]
        #Let all walkers set a random step
        Pos_new = Pos_walkers[:,i] + step_matrix_HO[:,i]*d_max1D
        
        #Determine acceptance probability 
        p = np.exp(-2 * alpha * Pos_new*Pos_new) * np.exp(2 * alpha * Pos_old*Pos_old)
        
        #Determine for each walkers if the step is accepted
        accept = np.where(p > accept_matrix_HO[:,i])
        not_accept = np.where(p <= accept_matrix_HO[:,i])
        
        #Move if new position is accepted
        Pos_walkers[accept,i+1] = Pos_new[accept]
        Pos_walkers[not_accept,i+1] = Pos_old[not_accept]

    #Calculate for all walkers, for each position the local energy EL and 
    #the local energy squared
    EL_array = Pos_walkers[:,step_equi:]*Pos_walkers[:,step_equi:]*(1/2 - 2*alpha**2) \
               + alpha
    EL2_array = EL_array * EL_array
    
    #Calculate the expectation value and variance of the energy 
    E = np.mean(EL_array)
    E2 = np.mean(EL2_array)
    Var = E2 - E**2
    
    print(alpha, end = "   ")
    print("%.4f" %E, end = "    ")
    print("%.5f"  %Var)

alpha      E      Var( E )
0.4   0.5129    0.02527
0.45   0.5030    0.00556
0.5   0.5000    0.00000
0.55   0.5022    0.00453
0.6   0.5081    0.01671


## The Helium Atom

As second step, we try and calculate the energy of the helium atom using VMC. The approach is explained in the book by Jos Thijssen (Chapter 12.2) and we compare the values with the table in the book. We use the same names for the parameters

### Initialize Parameters and random numbers

In [9]:
Nsteps = 20000   #Number of walker steps
N = 400          #Number of random walkers

#Parameter in wavefunction
alpha_HE = [0.05, 0.075, 0.10, 0.125, 0.15, 0.175, 0.2, 0.25]   

x_init_HE = np.random.uniform(-1, 1, (N,6)) #Unif. Initial position Random Walker
step_equi = 4000   #Number of equilibrium steps
 

#Matrix with random numbers, to accept (or not) the step
accept_matrix_HE = np.random.uniform(size = (N, Nsteps))  
#Matrix to give random stepsizes
step_matrix_HE = np.random.uniform(-1, 1, size = (N, 6, Nsteps))

#Maximal stepsize
d_max3D = 0.4

### Calculate the Energy, using Variational Monte Carlo

In [18]:
print('alpha      E      Var( E )   Acceptance probability')
for alpha in alpha_HE:
    
    #We set the initial position of the walkers to be random
    Pos_walkers = np.zeros((N, 6, Nsteps+1))
    Pos_walkers[:,:,0] = np.array(x_init_HE)
    
    #The matrix which will contain local energy at each point
    EL_matrix = np.zeros((N, Nsteps-step_equi))
    
    #Keep track of number of steps which is accepted
    num_accept = 0
    num_not_accept = 0
    
    #For each timestep we do the following:
    for i in range(Nsteps):

        #We calculate some distances, which are needed to calculate acceptance probability
        r1 = np.sqrt(np.sum(Pos_walkers[:,0:3,i]*Pos_walkers[:,0:3,i], axis = 1))
        r2 = np.sqrt(np.sum(Pos_walkers[:,3:6,i]*Pos_walkers[:,3:6,i], axis = 1))
        diff = Pos_walkers[:,3:6,i] - Pos_walkers[:,0:3,i]
        r12 = np.sqrt(np.sum(diff*diff, axis = 1))

        #Let all walkers set a random step
        Pos_new = Pos_walkers[:,:,i] + step_matrix_HE[:,:,i]*d_max3D
        
        #Calculate the same distances, after the steps
        r1_prime = np.sqrt(np.sum(Pos_new[:,0:3]*Pos_new[:,0:3], axis = 1))
        r2_prime = np.sqrt(np.sum(Pos_new[:,3:6]*Pos_new[:,3:6], axis = 1))
        diff_prime = Pos_new[:,3:6] - Pos_new[:,0:3]
        r12_prime = np.sqrt(np.sum(diff_prime*diff_prime, axis = 1))
        
        #Determine acceptance probability 
        p = np.exp(-4*(r1_prime-r1))*np.exp(-4*(r2_prime-r2))*np.exp(r12_prime/(1 
            + alpha*r12_prime))*np.exp(-r12/(1 + alpha*r12))
        
        #Determine for each walkers if the step is accepted
        accept = np.where(p > accept_matrix_HE[:,i])
        not_accept = np.where(p <= accept_matrix_HE[:,i])
        
        #Keep track of acceptance numbers
        num_accept += len(accept[0])
        num_not_accept += len(not_accept[0])
        
        #Move if new position is accepted
        Pos_walkers[accept,:, i+1] = Pos_new[accept]
        Pos_walkers[not_accept,:,i+1] = Pos_walkers[not_accept,:,i]

        #Now calculating the local energy, after equilibrium
        if i >= step_equi:
            
            #Calculate distances between electrons and atom
            r1_f = np.sqrt(np.sum(Pos_walkers[:,0:3,i+1]*Pos_walkers[:,0:3,i+1], axis = 1))
            r2_f = np.sqrt(np.sum(Pos_walkers[:,3:6,i+1]*Pos_walkers[:,3:6,i+1], axis = 1))
            diff_f = Pos_walkers[:,3:6,i+1] - Pos_walkers[:,0:3,i+1]
            r12_f = np.sqrt(np.sum(diff_f*diff_f, axis = 1))
            
            #Calculate one term of local energy
            b_term = r1_f + r2_f - np.sum(Pos_walkers[:,0:3,i+1]*Pos_walkers[:,3:6,i+1], 
                                          axis = 1) \
                    *(1/r1_f + 1/r2_f)
            
            #Calculate the local energy
            EL_array = -4 + b_term * 1/(r12_f*(1 + alpha*r12_f)**2) - 1/(r12_f*(1 + \
                        alpha*r12_f)**3) - 1/(4*(1 + alpha*r12_f)**4) + 1/r12_f

            EL_matrix[:,i-step_equi] = EL_array
    
    #Calculate the expectation value and variance of the energy 
    E = np.mean(EL_matrix)
    E2 = np.mean(EL_matrix*EL_matrix)
    Var = E2 - E**2
    accept_prob = num_accept/(num_accept + num_not_accept)
    
    print("%.3f" %alpha, end = "   ")
    print("%.4f" %E, end = "    ")
    print("%.5f"  %Var, end = "        ")
    print("%.3f" %accept_prob)

alpha      E      Var( E )   Acceptance probability
0.050   -2.8712    0.17530        0.582
0.075   -2.8753    0.15335        0.578
0.100   -2.8770    0.13598        0.575
0.125   -2.8779    0.12244        0.572
0.150   -2.8784    0.11155        0.569
0.175   -2.8779    0.10333        0.567
0.200   -2.8773    0.09694        0.565
0.250   -2.8751    0.08844        0.561
